![Logo_TraAM](https://raw.githubusercontent.com/PythonLycee/PyLyc/master/TraAM/Logo_TraAM.jpg)

<h2 style="color:#EA4786">TraAM 2021/2022</h2>
<h1 style="color:#2C81C1">Les mathématiques : moteur de l'intelligence artificielle</h1>
<h4 style="color:#EA4786">Académie de Strasbourg</h4>


<br><br>
<span style="color: #2C81C1"><i> Pour exécuter une saisie Python, sélectionner la cellule et valider avec  </span><span style="color: #EA4786"> <strong>SHIFT+Entrée</strong></i></span>.

<br><br>
<BLOCKQUOTE style="background-color:#FFDDDD">Ce fichier, pour l'instant à l'état de brouillon, est réalisé dans le cadre des TraAM 2021/2022.</BLOCKQUOTE>

<h1 style="color:#2C81C1; font-size:74px"> Le jeu de l'hexapion </h1>

### <span style="color:#2C81C1">Sommaire</span>

<ol style="color:#EA4786">
    <li> <a href="#1">Présentation du jeu de l'Hexapion</a></li>
    <li> <a href="#2">Étude exhaustive des coups possibles, stratégie(s) gagnante(s)</a></li>
    <li> <a href="#3">Programme complet </a></li>
    <li> <a href="#4">Activité en Spécialité NSI (Numérique et Sciences Informatiques)</a></li>
    <li> <a href="#5">Activité en Enseignement Scientifique </a></li>
    <li> <a href="#6">Sources et liens</a></li>
</ol>

<h1 style="color:#2C81C1" id="1">1. Présentation du jeu de l'Hexapion</h1>

<figure style="float: right;">
    <img src="https://raw.githubusercontent.com/PythonLycee/PyLyc/master/TraAM/pion.png" width="50%" height="50%">
</figure>

à venir

<h1 style="color:#2C81C1" id="2">2. Étude exhaustive des coups possibles, stratégie(s) gagnante(s)</h1>

Pour voir l'arbre des possibilités : <a href="https://github.com/PythonLycee/PyLyc/blob/e9a84237804f84d82d01b9baa49b8b764c9a6aa7/TraAM/hexapion_v3.pdf" target=_blank>Cliquer ici</a>

<h1 style="color:#2C81C1" id="3">3. Programme complet</h1>

<BLOCKQUOTE style="background-color:#FFDDDD; color:red;"><strong>AVERTISSEMENT :</strong> En raison de la gestion des input/print, l'exécution de la fonction <mark>partie</mark> ne fonctionne pas dans basthon. Il est préconisé de l'utiliser en local ou dans un serveur de type Binder.</BLOCKQUOTE>

<br><br>
Version provisoire (sans IA) : permet de jouer humain vs humain

In [1]:
N = 3 # largeur du plateau de jeu
M = 3 #longueur du plateau de jeu

#
#
# 0 1 ... N (<-ligne des noirs)
# 1
# .
# .
# M-1 ...   (<-ligne des blancs) 

from copy import deepcopy

class position:
    
    def __init__(self,grille=[[False]*N]+[[None]*N]*(M-2)+[[True]*N],tour_de_jeu=True):
        """
        Création d'une position.
        Par défaut, position de départ avec tour de jeu aux blancs
        (conventions : True = blanc, False = noir)
        """
        
        self.grille = grille 
        
        self.tour_de_jeu = tour_de_jeu
        
    def __eq__(self,other): #symbole ==
        """
        indique si deux positions sont identiques
        """
        return self.tour_de_jeu == other.tour_de_jeu and self.grille == other.grille
        
    def __invert__(self): #symbole ~ devant la position    
        """
        renvoie la position symétrique (avec même tour de jeu)
        """
        sym = position()
        return position([ ligne[::-1] for ligne in self.grille ],self.tour_de_jeu)
        
    def coups(self):
        """
        renvoie la liste des positions suivantes possibles
        """
        liste_coups=[]
        
        #on parcourt toutes les cases de la grille
        for m in range(len(self.grille)):
            for n in range(len(self.grille[m])):
                
                #variable pour le sens de mouvement
                sens = -1 if self.tour_de_jeu else 1
                #on repère les pions de la couleur qui joue
                
                if self.grille[m][n] == self.tour_de_jeu:

                    #on teste si le pion peut prendre ou avancer (trois directions possibles)
                    for dir in [-1,0,1]:
                        if 0 <= m+sens <M and 0<= n+dir <N and self.grille[m+sens][n+dir] == (None if dir==0 else not self.tour_de_jeu):
                            coup=position(deepcopy(self.grille),not self.tour_de_jeu) #on duplique la position (avec changement de tour de jeu)
                            coup.grille[m][n] = None #on vide la case du pion déplacé
                            coup.grille[m+sens][n+dir] = self.tour_de_jeu #on place le pion dans sa nouvelle position
                            liste_coups.append(coup) #on ajoute le coup à la liste des coups possibles
                    
        return liste_coups
        
    def gain(self):
        """
        renvoie le joueur gagnant:
        True -> blanc
        False -> noir
        None -> pas encore de vainqueur
        """
        
        #si un pion a atteint la dernière rangée, il y a un vainqueur
        if False in self.grille[M-1]: return False
        if True in self.grille[0]: return True
        
        #s'il n'y a plus de coups possibles, il y a un vainqueur
        #(NB: ce test gère aussi a fortiori le cas où il n'y a plus de jeton à jouer)
        if self.coups()==[]: return not self.tour_de_jeu
        
        #sinon il n'y a pas encore de vainqueur
        return None
        
    # méthode de classe provisoire pour l'affichage d'une position en console

    #    
    
    def genere_lignes_affichage(self,afftour=True):
        """
        génère une liste contenant chaque ligne d'affichage de la position dans la console
        "O" = pion blanc 
        " " = case vide
        "X" = pion noir
        ─┐┌└┘│
        
        Pour permettre une optimisation de l'affichage si on veut juxtaposer des grilles, 
        la méthode renvoie la liste des lignes (lignes sous forme de str)
        
        L'argument afftour permet de régler l'affichage ou non du tour de jeu
        """
        Aff_grille = ["┌───┐"]
        for ligne in self.grille:
            Aff_ligne = "│"
            for case in ligne:
                Aff_ligne += " " if case is None else "O" if case else "X"            
            Aff_ligne += "│"
            Aff_grille.append(Aff_ligne)
                
            
        Aff_grille.append("└───┘")
        if afftour:
            Aff_grille.append("Tour:") 
            Aff_grille.append("Blanc" if self.tour_de_jeu else "Noir ")
        return Aff_grille  
        
    def __str__(self):
        """
        Conversion en chaîne de caractère
        Permet d'utiliser la syntaxe print pour un objet position
        """    
        Aff=""
        for ligne in self.genere_lignes_affichage(): Aff += ligne+"\n"
        return Aff
    
        

def partie():
    
    pos = position() # génération de la position initiale (début de partie)
    
    while pos.gain() is None:
                
        coups_possibles = pos.coups() # génération de la liste des coups possibles
        
        Message="\nPosition actuelle\n"+str(pos)+"\n___________________\nChoisir un coup:\n"
        
        
        for k in range(len(coups_possibles)):
            Message += "\n"+str(k)+"    "
            for ligne in coups_possibles[k].genere_lignes_affichage(): Message += ligne+"\n"
        
        while True:
            try:
                choix = input(Message)
                pos = coups_possibles[int(choix)]
                Message=""
                break
            except:
                Message+="Saisie invalide.\n"
        
        print("\n_________________________________\nPosition finale:\n")
        print(pos)    
        
    return "Gain " + ("blanc" if pos.gain() else "noir")
            



In [13]:
N = 3 # largeur du plateau de jeu
M = 3 #longueur du plateau de jeu

#
#
# 0 1 ... N (<-ligne des noirs)
# 1
# .
# .
# M-1 ...   (<-ligne des blancs) 

from copy import deepcopy

class position:
    
    def __init__(self,grille=[[False]*N]+[[None]*N]*(M-2)+[[True]*N],tour_de_jeu=True):
        """
        Création d'une position.
        Par défaut, position de départ avec tour de jeu aux blancs
        (conventions : True = blanc, False = noir)
        """
        
        self.grille = grille 
        
        self.tour_de_jeu = tour_de_jeu
        
    def __eq__(self,other): #symbole ==
        """
        indique si deux positions sont identiques
        """
        return self.tour_de_jeu == other.tour_de_jeu and self.grille == other.grille
        
    def __invert__(self): #symbole ~ devant la position    
        """
        renvoie la position symétrique (avec même tour de jeu)
        """
        sym = position()
        return position([ ligne[::-1] for ligne in self.grille ],self.tour_de_jeu)
        
    def coups(self):
        """
        renvoie la liste des positions suivantes possibles
        """
        liste_coups=[]
        
        #on parcourt toutes les cases de la grille
        for m in range(len(self.grille)):
            for n in range(len(self.grille[m])):
                
                #variable pour le sens de mouvement
                sens = -1 if self.tour_de_jeu else 1
                #on repère les pions de la couleur qui joue
                
                if self.grille[m][n] == self.tour_de_jeu:

                    #on teste si le pion peut prendre ou avancer (trois directions possibles)
                    for dir in [-1,0,1]:
                        if 0 <= m+sens <M and 0<= n+dir <N and self.grille[m+sens][n+dir] == (None if dir==0 else not self.tour_de_jeu):
                            coup=position(deepcopy(self.grille),not self.tour_de_jeu) #on duplique la position (avec changement de tour de jeu)
                            coup.grille[m][n] = None #on vide la case du pion déplacé
                            coup.grille[m+sens][n+dir] = self.tour_de_jeu #on place le pion dans sa nouvelle position
                            liste_coups.append(coup) #on ajoute le coup à la liste des coups possibles
                    
        return liste_coups
        
    def gain(self):
        """
        renvoie le joueur gagnant:
        True -> blanc
        False -> noir
        None -> pas encore de vainqueur
        """
        
        #si un pion a atteint la dernière rangée, il y a un vainqueur
        if False in self.grille[M-1]: return False
        if True in self.grille[0]: return True
        
        #s'il n'y a plus de coups possibles, il y a un vainqueur
        #(NB: ce test gère aussi a fortiori le cas où il n'y a plus de jeton à jouer)
        if self.coups()==[]: return not self.tour_de_jeu
        
        #sinon il n'y a pas encore de vainqueur
        return None
        
    # méthode de classe provisoire pour l'affichage d'une position en console

    #    
    
    def genere_lignes_affichage(self,afftour=True):
        """
        génère une liste contenant chaque ligne d'affichage de la position dans la console
        "O" = pion blanc 
        " " = case vide
        "X" = pion noir
        ─┐┌└┘│
        
        Pour permettre une optimisation de l'affichage si on veut juxtaposer des grilles, 
        la méthode renvoie la liste des lignes (lignes sous forme de str)
        
        L'argument afftour permet de régler l'affichage ou non du tour de jeu
        """
        Aff_grille = ["┌───┐"]
        for ligne in self.grille:
            Aff_ligne = "│"
            for case in ligne:
                Aff_ligne += " " if case is None else "O" if case else "X"            
            Aff_ligne += "│"
            Aff_grille.append(Aff_ligne)
                
            
        Aff_grille.append("└───┘")
        if afftour:
            Aff_grille.append("Tour:") 
            Aff_grille.append("Blanc" if self.tour_de_jeu else "Noir ")
        return Aff_grille  
        
    def __str__(self):
        """
        Conversion en chaîne de caractère
        Permet d'utiliser la syntaxe print pour un objet position
        """    
        Aff=""
        for ligne in self.genere_lignes_affichage(): Aff += ligne+"\n"
        return Aff
    
        

def partie():
    
    pos = position() # génération de la position initiale (début de partie)
    
    while pos.gain() is None:
                
        coups_possibles = pos.coups() # génération de la liste des coups possibles
        
        Message="\nPosition actuelle\n"+str(pos)+"\n___________________\nChoisir le coup "+("blanc" if pos.tour_de_jeu else "noir")     +":\n"
        
        for k in range(len(coups_possibles)):
            Message += str(k)+"     "
        Message += "\n"
        
        m=["" for k in range(5)]        
            
        for coup in coups_possibles:
            for k in range(5):
                m[k] += coup.genere_lignes_affichage(afftour=False)[k]+" "
                
        for k in range(5):
            Message += m[k]+"\n"
        
        
        while True:
            try:
                choix = input(Message)
                pos = coups_possibles[int(choix)]
                Message=""
                break
            except:
                Message+="Saisie invalide.\n"
        
        print("\n_________________________________\nPosition finale:\n")
        print(pos)    
        
    return "Gain " + ("blanc" if pos.gain() else "noir")
            



In [14]:
# Exécuter cette cellule pour jouer une partie
partie()


Position actuelle
┌───┐
│XXX│
│   │
│OOO│
└───┘
Tour:
Blanc

___________________
Choisir le coup blanc:
0     1     2     
┌───┐ ┌───┐ ┌───┐ 
│XXX│ │XXX│ │XXX│ 
│O  │ │ O │ │  O│ 
│ OO│ │O O│ │OO │ 
└───┘ └───┘ └───┘ 
0

_________________________________
Position finale:

┌───┐
│XXX│
│O  │
│ OO│
└───┘
Tour:
Noir 


Position actuelle
┌───┐
│XXX│
│O  │
│ OO│
└───┘
Tour:
Noir 

___________________
Choisir le coup noir:
0     1     2     
┌───┐ ┌───┐ ┌───┐ 
│X X│ │X X│ │XX │ 
│X  │ │OX │ │O X│ 
│ OO│ │ OO│ │ OO│ 
└───┘ └───┘ └───┘ 
1

_________________________________
Position finale:

┌───┐
│X X│
│OX │
│ OO│
└───┘
Tour:
Blanc


Position actuelle
┌───┐
│X X│
│OX │
│ OO│
└───┘
Tour:
Blanc

___________________
Choisir le coup blanc:
0     1     
┌───┐ ┌───┐ 
│X X│ │X X│ 
│OO │ │OXO│ 
│ O │ │ O │ 
└───┘ └───┘ 
0

_________________________________
Position finale:

┌───┐
│X X│
│OO │
│ O │
└───┘
Tour:
Noir 


Position actuelle
┌───┐
│X X│
│OO │
│ O │
└───┘
Tour:
Noir 

___________________
Choi

'Gain blanc'

<h1 style="color:#2C81C1" id="4">4. Activité en Spécialité NSI (Numérique et Sciences Informatiques)</h1>

à venir

<h1 style="color:#2C81C1" id="5">5. Activité en Enseignement Scientifique</h1>

à venir

<h1 style="color:#2C81C1" id="6">6. Sources et liens</h1>
<br><br>
<strong>Sources :</strong>
<ul>
    <li><a href="https://etincelle.blog/2021/03/18/un-jeu-a-construire-soi-meme-pour-tout-comprendre-sur-lintelligence-artificielle/ ">https://etincelle.blog/2021/03/18/un-jeu-a-construire-soi-meme-pour-tout-comprendre-sur-lintelligence-artificielle/ </a></li> 
    <li><a href="https://www.wikidebrouillard.org/wiki/Intelligence_artificielle_DIY_imbatable_%C3%A0_l%27hexapion">https://www.wikidebrouillard.org/wiki/Intelligence_artificielle_DIY_imbatable_%C3%A0_l%27hexapion </a></li>
    <li><a href="https://files.inria.fr/mecsci/classcodeIAI/pdf/ressources_complementaires/m2_IAI_fichepedagosup-Hexapion.pdf">https://files.inria.fr/mecsci/classcodeIAI/pdf/ressources_complementaires/m2_IAI_fichepedagosup-Hexapion.pdf </a></li>
</ul>
